# Forecasting Consensus Expectations: Nonfarm Payrolls (NFP) 

## Data preprocessing

**Imports**

In [73]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from glob import glob

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera

**Helper functions**

In [74]:
def load_hist_nfp(path: str) -> pd.DataFrame:
    """
    Loads and pivots Bloomberg's historical NFP data to a long dataframe format (one row per economist forecast).
    """
    raw = pd.read_excel(path, header=None, engine="openpyxl")
    name_row, firm_row = raw.iloc[1], raw.iloc[2]

    # find summary columns once
    col_actual = name_row.str.contains("Actual", case=False, na=False).idxmax()
    col_median = name_row.str.contains("Median", case=False, na=False).idxmax()

    dates  = pd.to_datetime(raw.iloc[3:, 0])
    period = pd.to_datetime(raw.iloc[3:, 1])
    actual = pd.to_numeric(raw.iloc[3:, col_actual], errors="coerce")
    median = pd.to_numeric(raw.iloc[3:, col_median], errors="coerce")

    base = pd.DataFrame(
        {"release_date": dates,
         "period":       period,
         "actual":       actual,
         "median_forecast": median}
    )

    skip_tokens = {"median", "average", "survey", "high",
                   "low", "previous", "prior", "actual"}
    frames = []
    ASOF_FMT = "%m/%d/%Y"

    for col in range(2, raw.shape[1]):
        hdr = name_row[col]
        if (not isinstance(hdr, str)
            or any(tok in hdr.lower() for tok in skip_tokens)):
            continue

        tmp = base.copy()
        tmp["economist"] = hdr
        tmp["firm"]      = firm_row[col]
        tmp["forecast"]  = pd.to_numeric(raw.iloc[3:, col], errors="coerce")
        tmp["asof"]      = pd.to_datetime(raw.iloc[3:, col-1],
                                          format=ASOF_FMT, errors="coerce")
        frames.append(tmp)

    return (pd.concat(frames, ignore_index=True)
              .dropna(subset=["forecast"]))


In [75]:
def load_single_release(raw_path: str,
                        release_date: str,
                        period_start: str) -> pd.DataFrame:
    """
    Converts a single Bloomberg NFP release into the long table layout for merge.
    """
    raw = pd.read_excel(raw_path, sheet_name=0, header=None,
                        engine="openpyxl")

    # --- forecaster block ------------------------------------------------
    fcst_num = pd.to_numeric(raw[3], errors="coerce")
    mask = raw[0].isna() & fcst_num.notna()

    block = (raw.loc[mask, [1, 2, 3, 4, 5]]
               .rename(columns={1: "economist",
                                2: "firm",
                                3: "forecast",
                                4: "asof",
                                5: "rank"}))

    block["forecast"] = fcst_num[mask]
    block["asof"]     = pd.to_datetime(block["asof"], errors="coerce")

    # fallback: if economist name missing, use firm in parentheses
    block["economist"] = block.apply(
        lambda r: r["economist"] if pd.notna(r["economist"])
        else f"({r['firm']})", axis=1
    )

    # --- actual print ----------------------------------------------------
    actual_mask = raw.apply(
        lambda r: r.astype(str).str.contains("Actual", case=False).any(),
        axis=1
    )
    actual_val = (pd.to_numeric(raw.loc[actual_mask].stack(),
                                errors="coerce").dropna()
                  .iloc[0] if actual_mask.any() else np.nan)

    rel_date = pd.to_datetime(release_date)
    if rel_date > pd.Timestamp.today().normalize():
        actual_val = np.nan       # future month – unknown actual yet

    # --- tidy output -----------------------------------------------------
    tidy = (block.assign(release_date=rel_date,
                         period=pd.to_datetime(period_start),
                         actual=actual_val)
                 .loc[:, ["release_date", "period", "economist",
                          "firm", "forecast", "actual", "asof"]])

    return tidy


In [76]:
def build_nfp_long(hist_path: str,
                   singles: list[tuple[str, str, str]] | None = None
) -> pd.DataFrame:
    """
    Simple helper for concatenating single releases with historical NFP data.
    """

    nfp_long = load_hist_nfp(hist_path)

    if singles:
        for fp, rdate, pstart in singles:
            if os.path.exists(fp):
                nfp_long = pd.concat(
                    [nfp_long,
                     load_single_release(fp, rdate, pstart)],
                    ignore_index=True)
    return nfp_long


**Build NFP long dataframe + run checks**

In [77]:
# Example single release tuple format, (file_path, release_date, observation_date)
# SINGLE_RELEASES: list[tuple[str, str, str]] = [
#     # ("../raw/nfp_aug25.xlsx", "2025-09-05", "2025-08-01"),
# ]

In [78]:
HIST_PATH = "../raw/nfp_historical.xlsx"


SINGLE_RELEASES = []        # specify single releases here if applicable, format as above
nfp_long = build_nfp_long(HIST_PATH, singles=SINGLE_RELEASES)
nfp_long.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof
367,1998-12-04,1998-11-30,267.0,160.0,Peter Dixon,NaN,160.0,1998-12-04
368,1999-01-08,1998-12-31,378.0,200.0,Peter Dixon,NaN,200.0,1999-01-08
369,1999-02-05,1999-01-31,245.0,140.0,Peter Dixon,NaN,140.0,1999-02-05
370,1999-03-05,1999-02-28,275.0,245.0,Peter Dixon,NaN,220.0,1999-03-05
371,1999-04-02,1999-03-31,46.0,150.0,Peter Dixon,NaN,160.0,1999-03-29


In [79]:
# sort and keep last forecast per economist
nfp_last = (nfp_long
            .sort_values(["release_date", "economist", "asof"])
            .groupby(["release_date", "economist"], as_index=False)
            .tail(1)
            .reset_index(drop=True))

# nfp surprise
nfp_last["surprise"] = nfp_last["actual"] - nfp_last["median_forecast"]

nfp_last.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof,surprise
0,1997-08-01,1997-07-31,316.0,70.0,Anthony Chan,JPMorgan Chase Bank,75.0,1997-08-01,246.0
1,1997-08-01,1997-07-31,316.0,70.0,Bruce Steinberg,Merrill Lynch & Co Inc,55.0,1997-08-01,246.0
2,1997-08-01,1997-07-31,316.0,70.0,Crandall/Jordan,Wrightson ICAP LLC,125.0,1997-08-01,246.0
3,1997-08-01,1997-07-31,316.0,70.0,David H Resler,Nomura Securities Intl Inc,65.0,1997-08-01,246.0
4,1997-08-01,1997-07-31,316.0,70.0,David J Greenlaw,Morgan Stanley & Co LLC,100.0,1997-08-01,246.0


In [80]:
# sanity checks

# {release_date, economist} pairs should be unique
assert nfp_last.duplicated(["release_date", "economist"]).sum() == 0

# asof has to strictly precede or be equal to release date 
valid_asof = nfp_last["asof"].notna()
assert (nfp_last.loc[valid_asof, "asof"]
        <= nfp_last.loc[valid_asof, "release_date"]).all()

# should have no more than one unique realized print per date 
assert (nfp_last.groupby("release_date")["actual"].nunique() <= 1).all()

**Export**

In [81]:
# ---------------------------------------------------------------------
# Output
# ---------------------------------------------------------------------
import os
OUT_DIR       = "../out"
DF_FILE       = "nfp_df.parquet"        # filtered (post‑COVID exclusions)
DF_FULL_FILE  = "nfp_df_full.parquet"   # full history

os.makedirs(OUT_DIR, exist_ok=True)

# covid‐adjusted subset (starts 2003‑06, drops 2020‑2022)
df = (nfp_last
      .query("release_date >= '2003-06-01'")
      .query("~release_date.between('2020-01-01', '2022-12-31')"))

df.to_parquet(os.path.join(OUT_DIR, DF_FILE),
              engine="pyarrow", index=False)
nfp_last.to_parquet(os.path.join(OUT_DIR, DF_FULL_FILE),
                    engine="pyarrow", index=False)

print(f"Saved clean  ➜  {OUT_DIR}/{DF_FILE}")
print(f"Saved full   ➜  {OUT_DIR}/{DF_FULL_FILE}")

Saved clean  ➜  ../out/nfp_df.parquet
Saved full   ➜  ../out/nfp_df_full.parquet
